In [1]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
import subprocess
from docx.shared import Pt
import pandas as pd
import os
import shutil
from tqdm import tqdm

In [2]:
consult = pd.read_excel('Input/04. Report PPU April 2024.xlsx', sheet_name='Discharge Consult')

/opt/conda/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
#rx = pd.read_excel('Input/04. Report PPU April 2024.xlsx', sheet_name='Discharge Rx')

In [4]:
consult.columns = consult.columns.str.lower().str.strip().str.replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]','', regex = True).str.replace(' ', '_')

In [5]:
consult_mrg = pd.read_excel('Input/sample_consult_file_w_prescription_3.xlsx')


In [12]:
#consult.columns = consult.columns.str.lower().str.strip().str.replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]','', regex = True).str.replace(' ', '_')
#rx.columns = rx.columns.str.lower().str.strip().str.replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]','', regex = True).str.replace(' ', '_')
consult_mrg.head()

,Unnamed: 0,Date,Name,Card No,GDT ID,Consult ID,ICDX,GP/SP,Claim ID,Consult Fee,...,UNIT OBAT 8,UNIT OBAT 9,UNIT OBAT 10,UNIT OBAT 11,UNIT OBAT 12,Consultation ID,Doctor SIP,Doctor STR,Chief Complaints,Suggestion
0,0,2024-04-27,Afriliana Berdi,8000150301320947,7093643,1401659,"S90, P12.3",GP,133797350 & 133802422,29970.0,...,Tablet,Tablet,Tablet,NaN,NaN,1401659,440/0144/DU/DPM-PTSP.PPJU/OL.22,3111100120211971,Kaki saya keserempet motor udah di pijit agak ...,Kompres air dingin dan pakai obat secara teratur
1,1,2024-04-27,Agung Budhi Cahyono,8000150301293037,6673283,1402097,J00,GP,133810311 & 133811616,29970.0,...,Tablet,Tablet,Tablet,NaN,NaN,1402097,3/B.15A/31.71.08.1001.03.003.R.4/3/-1.779.3/e/...,3111100221157260,"saya Agak Demam, Batuk, tenggorokan Serak",tingkatkan daya tahan tubuh dengan: makan yang...
2,2,2024-04-27,Anang Rahyudi Budiantoro,8000150301336844,2361628,1402119,R73.9,GP,133810512,29970.0,...,Tablet,Tablet,Tablet,NaN,NaN,1402119,KS.08/4904/DPMPTSP/DU/2003,3111100223202441,selamat sore dok..\nsya tadi dikantor ikut cek...,"hindari makan / minuman manis ya, terutama yan..."
3,3,2024-04-27,Ayuni Zayna Alfathunnisa,8000150301311268,2175215,1401330,J00,GP,133784230,29970.0,...,Tablet,Tablet,Tablet,NaN,NaN,1401330,440/0636/DU/DPM-PTSP.PPJU/OL.21,3221100221180606,"Batuk, radang tenggorokam, demam,",NaN
4,4,2024-04-27,Farzana Shafira Almahira,8000150301269417,6291767,1401484,L23,SP,133784568 & 133791596,111000.0,...,Tablet,Tablet,Tablet,NaN,NaN,1401484,503/DPMPTSP/Sp/2020,7321602217116862,saya mengalami gigitan serangga,Kontrol 7 hari jika ada keluhan


In [13]:
#rx[rx['Consultation ID'].isin(consult['Consultation ID']) == False].dropna(subset = 'Order Created Date')
#consult_mrg = consult.merge(rx, how = 'left', on = 'consultation_id', suffixes=('', '_rx')).fillna('')

In [14]:
consult_mrg.columns = consult_mrg.columns.str.lower().str.strip().str.replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]','', regex = True).str.replace(' ', '_')
consult_mrg = consult_mrg.merge(consult[['consultation_id','dob','gender']])

In [15]:
data = consult_mrg.iloc[3465].to_dict()

In [19]:
#consult_mrg.shape

In [21]:
def row_to_pdf(data):
    for i in data.keys():
        if str(data.get(i)) == 'nan':
            data[i] = ''
            
    f = open('template.docx', 'rb')
    doc = Document(f)
    
    
    ## General Information
    doc.tables[0].cell(0, 1).text = data['doctor_name']
    doc.tables[0].cell(1, 1).text = data['doctor_department']
    doc.tables[0].cell(2, 1).text = data['doctor_sip']
    doc.tables[0].cell(3, 1).text = data['doctor_str']
    doc.tables[0].cell(0, 3).text = data['name']
    doc.tables[0].cell(1, 3).text = data['gender']
    doc.tables[0].cell(2, 3).text = data['dob']
    doc.tables[0].cell(3, 3).text = str(data['card_no']).replace('.0', '')
    doc.tables[0].cell(4, 3).text = data['payor']
    doc.tables[0].cell(5, 3).text = data['corporate']
    
    for i in range(6):
        if i < 4:
            doc.tables[0].cell(i, 1).paragraphs[0].runs[0].font.size = Pt(8)
        doc.tables[0].cell(i, 3).paragraphs[0].runs[0].font.size = Pt(8)

    ## complaint & diagnosis
    doc.paragraphs[2].text = data['chief_complaints']
    doc.paragraphs[2].runs[0].font.size= Pt(8)
    doc.paragraphs[4].text = str(data['diagnosis']) if str(data['diagnosis']) != '' else '' + (', ' + str(data['suggestion'])) if str(data['suggestion']) != '' else '' 
    doc.paragraphs[4].runs[0].font.size= Pt(8)
    
    doc.paragraphs[6].text = data['icdx']
    doc.paragraphs[6].runs[0].font.size= Pt(8)

    ## Consultation Detail
    doc.tables[1].cell(1, 1).text = str(data['consult_id']).replace('.0', '')
    doc.tables[1].cell(1, 2).text = str(data['claim_id']).replace('.0', '')
    doc.tables[1].cell(1, 3).text = data['date'].strftime('%d/%m/%Y') + ' ' + data['time']
    doc.tables[1].cell(1, 4).text = 'IDR '+ str(data['consult_fee']).replace('.0', '')
    doc.tables[1].cell(1, 7).text = 'IDR '+ str(data['consult_fee']).replace('.0', '')
    doc.tables[1].cell(2, 1).text = str(data['order_id']).replace('.0', '')
    
    
    doc.tables[1].cell(1, 1).paragraphs[0].runs[0].font.size= Pt(8)
    doc.tables[1].cell(1, 2).paragraphs[0].runs[0].font.size= Pt(8)
    doc.tables[1].cell(1, 3).paragraphs[0].runs[0].font.size= Pt(8)
    doc.tables[1].cell(1, 4).paragraphs[0].runs[0].font.size= Pt(8)
    doc.tables[1].cell(1, 7).paragraphs[0].runs[0].font.size= Pt(8)
    doc.tables[1].cell(2, 1).paragraphs[0].runs[0].font.size= Pt(8)

    for i in range(1,13):
        j = i + 2
        if str(data[f'obat_{i}']) != '':
            doc.tables[1].cell(j, 0).text = data[f'obat_{i}']
            doc.tables[1].cell(j, 0).paragraphs[0].alignment = 2
            doc.tables[1].cell(j, 4).text = 'IDR '+ str(data[f'total_{i}']).replace('.0', '')
            doc.tables[1].cell(j, 5).text = str(data[f'jumlah_{i}']).replace('.0', '')
            doc.tables[1].cell(j, 6).text = data[f'unit_obat_{i}']
            doc.tables[1].cell(j, 7).text = 'IDR '+ str(data[f'total_{i}']).replace('.0', '')

            doc.tables[1].cell(j, 6).paragraphs[0].runs[0].italic = True
            
            doc.tables[1].cell(j, 0).paragraphs[0].runs[0].font.size= Pt(8)
            doc.tables[1].cell(j, 4).paragraphs[0].runs[0].font.size= Pt(8)
            doc.tables[1].cell(j, 5).paragraphs[0].runs[0].font.size= Pt(8)
            doc.tables[1].cell(j, 6).paragraphs[0].runs[0].font.size= Pt(8)
            doc.tables[1].cell(j, 7).paragraphs[0].runs[0].font.size= Pt(8)

    doc.tables[1].cell(16, 4).text = 'IDR 0' if data['excess_delivery_fee'] =='' else 'IDR '+ str(data['excess_delivery_fee']).replace('.0', '')
    doc.tables[1].cell(16, 7).text = 'IDR 0' if data['excess_delivery_fee'] =='' else 'IDR '+ str(data['excess_delivery_fee']).replace('.0', '')
    doc.tables[1].cell(18, 7).text = 'IDR '+ str(data['total']).replace('.0', '')
    excess =  0 if  data['rx_excess'] == '' else float(data['rx_excess']) + 0 if  data['excess_consult'] == '' else data['excess_consult']
    doc.tables[1].cell(19, 7).text = 'IDR '+ str(data['total'] - excess).replace('.0', '')
    doc.tables[1].cell(20, 7).text = 'IDR '+ str(excess).replace('.0', '')
    
    doc.tables[1].cell(16, 4).paragraphs[0].runs[0].font.size= Pt(8)
    doc.tables[1].cell(16, 7).paragraphs[0].runs[0].font.size= Pt(8)
    doc.tables[1].cell(18, 7).paragraphs[0].runs[0].font.size= Pt(8)
    doc.tables[1].cell(19, 7).paragraphs[0].runs[0].font.size= Pt(8)
    doc.tables[1].cell(20, 7).paragraphs[0].runs[0].font.size= Pt(8)
    
    doc.tables[1].cell(18, 7).paragraphs[0].runs[0].bold = True
    doc.tables[1].cell(19, 7).paragraphs[0].runs[0].bold = True
    doc.tables[1].cell(20, 7).paragraphs[0].runs[0].bold = True

    name = data['name'].replace('/', ' ')
    consult_on = data['date'].strftime('%Y-%m-%d')
    consult_id = str(data['consult_id']).replace('.0', '')
    agg = data['aggregator_name']
    payor = data['payor']
    
    if not os.path.exists(f'.tmp/{agg}/{payor}'):
        os.makedirs(f'.tmp/{agg}/{payor}')

    tmp_name = f'.tmp/{agg}/{payor}/{consult_on}_Consultation_Receipt_{name}_{consult_id}.docx'
    doc.save(tmp_name)
    f.close()
    
    subprocess.run(['libreoffice', '--convert-to', 'pdf' ,
                    tmp_name, '--outdir', f'.tmp/{agg}/{payor}']
                   ,stdout=subprocess.DEVNULL,
                    stderr=subprocess.DEVNULL
                )

    #old_doc = [x for x in os.listdir('output') if 'docx' in x]
    #for i in old_doc:
    os.remove(tmp_name)

In [34]:
shutil.rmtree('.tmp')
os.mkdir('.tmp')

shutil.rmtree('output')
os.mkdir('output')

In [32]:
consult_mrg = consult_mrg.sort_values(['aggregator_name', 'payor'], ascending = False)
payor_stat = consult_mrg['payor'].value_counts().reset_index()
pyr = payor_stat[payor_stat['count'] < 300].iloc[:3].payor

In [ ]:
%%time
for index, row in tqdm(consult_mrg.iterrows()):
    row_to_pdf(row)

12600it [54:43:23,  1.50s/it]  

In [ ]:
consult_mrg

In [75]:
for i in data.keys():
    if str(data.get(i)) == 'nan':
        data[i] = ''

In [76]:
data['ag']

''

In [77]:
f = open('template.docx', 'rb')
doc = Document(f)

In [79]:
doc.tables[0].cell(0, 1).text = data['doctor_name']
doc.tables[0].cell(1, 1).text = data['doctor_department']
doc.tables[0].cell(2, 1).text = data['doctor_sip']
doc.tables[0].cell(3, 1).text = data['doctor_str']
doc.tables[0].cell(0, 3).text = data['name']
doc.tables[0].cell(1, 3).text = data['gender']
doc.tables[0].cell(2, 3).text = data['dob']
doc.tables[0].cell(3, 3).text = str(data['card_no']).replace('.0', '')
doc.tables[0].cell(4, 3).text = data['payor']
doc.tables[0].cell(5, 3).text = data['corporate']

In [80]:
for i in range(6):
    if i < 4:
        doc.tables[0].cell(i, 1).paragraphs[0].runs[0].font.size = Pt(8)
    doc.tables[0].cell(i, 3).paragraphs[0].runs[0].font.size = Pt(8)

In [81]:
doc.paragraphs[2].text = data['chief_complaints']
doc.paragraphs[2].runs[0].font.size= Pt(8)

In [91]:
doc.paragraphs[4].text = str(data['diagnosis']) if str(data['diagnosis']) != '' else '' + (', ' + str(data['suggestion'])) if str(data['suggestion']) != '' else '' 
doc.paragraphs[4].runs[0].font.size= Pt(8)

In [92]:
doc.paragraphs[6].text = data['icdx']
doc.paragraphs[6].runs[0].font.size= Pt(8)

doc.tables[1].cell(1, 1).text = str(data['consult_id']).replace('.0', '')
doc.tables[1].cell(1, 2).text = str(data['claim_id']).replace('.0', '')
doc.tables[1].cell(1, 3).text = data['date'].strftime('%d/%m/%Y') + ' ' + data['time']
doc.tables[1].cell(1, 4).text = 'IDR '+ str(data['consult_fee']).replace('.0', '')
doc.tables[1].cell(1, 7).text = 'IDR '+ str(data['consult_fee']).replace('.0', '')
doc.tables[1].cell(2, 1).text = str(data['order_id']).replace('.0', '')


doc.tables[1].cell(1, 1).paragraphs[0].runs[0].font.size= Pt(8)
doc.tables[1].cell(1, 2).paragraphs[0].runs[0].font.size= Pt(8)
doc.tables[1].cell(1, 3).paragraphs[0].runs[0].font.size= Pt(8)
doc.tables[1].cell(1, 4).paragraphs[0].runs[0].font.size= Pt(8)
doc.tables[1].cell(1, 7).paragraphs[0].runs[0].font.size= Pt(8)
doc.tables[1].cell(2, 1).paragraphs[0].runs[0].font.size= Pt(8)




In [93]:
#doc.tables[1].cell(1, 1).paragraphs[0].runs

In [94]:
for i in range(1,13):
    j = i + 2
    if str(data[f'obat_{i}']) != '':
        doc.tables[1].cell(j, 0).text = data[f'obat_{i}']
        doc.tables[1].cell(j, 0).paragraphs[0].alignment = 2
        doc.tables[1].cell(j, 5).text = str(data[f'jumlah_{i}']).replace('.0', '')
        doc.tables[1].cell(j, 6).text = data[f'unit_obat_{i}']
        doc.tables[1].cell(j, 7).text = 'IDR '+ str(data[f'total_{i}']).replace('.0', '')
        
        doc.tables[1].cell(j, 0).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(j, 5).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(j, 6).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(j, 7).paragraphs[0].runs[0].font.size= Pt(8)


In [99]:
doc.tables[1].cell(16, 7).text = 'IDR 0' if data['excess_delivery_fee'] == '' else 'IDR '+ str(data['excess_delivery_fee']).replace('.0', '')
doc.tables[1].cell(18, 7).text = 'IDR '+ str(data['total']).replace('.0', '')
excess = float(data['rx_excess']) - 0 if  data['excess_consult'] == '' else data['excess_consult']
doc.tables[1].cell(19, 7).text = 'IDR '+ str(data['total'] - excess).replace('.0', '')
doc.tables[1].cell(20, 7).text = 'IDR '+ str(excess).replace('.0', '')


doc.tables[1].cell(16, 7).paragraphs[0].runs[0].font.size= Pt(8)
doc.tables[1].cell(18, 7).paragraphs[0].runs[0].font.size= Pt(8)
doc.tables[1].cell(19, 7).paragraphs[0].runs[0].font.size= Pt(8)
doc.tables[1].cell(20, 7).paragraphs[0].runs[0].font.size= Pt(8)

doc.tables[1].cell(18, 7).paragraphs[0].runs[0].bold = True
doc.tables[1].cell(19, 7).paragraphs[0].runs[0].bold = True
doc.tables[1].cell(20, 7).paragraphs[0].runs[0].bold = True

In [100]:
doc.save(f'test.docx')
f.close()

Question:
* apakah ada footer?
* rx sm consult claim_id bisa beda?
* Unit utk konsultasi sm delivery mau pake sesi aja?
* delivery fee itu excess_delivery_fee bukan?
* Nama Insurance pake apa?

In [101]:
data['payor']

'AMAG'